In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix

In [4]:
df = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')

In [11]:
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [13]:
df.shape

(8124, 23)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [14]:
df.columns

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')

In [15]:
for col in df.columns:
    print(col,': ', df[col].unique())
    print()

class :  ['p' 'e']

cap-shape :  ['x' 'b' 's' 'f' 'k' 'c']

cap-surface :  ['s' 'y' 'f' 'g']

cap-color :  ['n' 'y' 'w' 'g' 'e' 'p' 'b' 'u' 'c' 'r']

bruises :  ['t' 'f']

odor :  ['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']

gill-attachment :  ['f' 'a']

gill-spacing :  ['c' 'w']

gill-size :  ['n' 'b']

gill-color :  ['k' 'n' 'g' 'p' 'w' 'h' 'u' 'e' 'b' 'r' 'y' 'o']

stalk-shape :  ['e' 't']

stalk-root :  ['e' 'c' 'b' 'r' '?']

stalk-surface-above-ring :  ['s' 'f' 'k' 'y']

stalk-surface-below-ring :  ['s' 'f' 'y' 'k']

stalk-color-above-ring :  ['w' 'g' 'p' 'n' 'b' 'e' 'o' 'c' 'y']

stalk-color-below-ring :  ['w' 'p' 'g' 'b' 'n' 'e' 'y' 'o' 'c']

veil-type :  ['p']

veil-color :  ['w' 'n' 'o' 'y']

ring-number :  ['o' 't' 'n']

ring-type :  ['p' 'e' 'l' 'f' 'n']

spore-print-color :  ['k' 'n' 'u' 'h' 'w' 'r' 'o' 'y' 'b']

population :  ['s' 'n' 'a' 'v' 'y' 'c']

habitat :  ['u' 'g' 'm' 'd' 'p' 'w' 'l']



In [16]:
#Since veil-type has single value so we can remove this column
df.drop('veil-type', axis=1, inplace=True)

In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [22]:
for col in df.columns:
    df[col]=le.fit_transform(df[col])

In [19]:
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,2,7,7,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,2,7,7,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,2,7,7,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,2,7,7,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,2,7,7,2,1,0,3,0,1


In [23]:
X = df.drop('class', axis=1)
y = df['class']

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
 X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=102)

In [23]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [24]:
lr.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [25]:
y_pred = lr.predict(X_test)

In [26]:
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix

In [27]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1342,   51],
       [  75, 1213]])

In [28]:
sc = f1_score(y_test, y_pred)
sc

0.9506269592476491

In [29]:
rs = recall_score(y_test, y_pred)
rs

0.9417701863354038

In [30]:
ps = precision_score(y_test, y_pred)
ps

0.9596518987341772

# Hyperparameter Tuning

In [15]:
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV

# 1. Tuning logistic regression

In [32]:
# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

In [33]:
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=20, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)

In [34]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.967463 using {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.967463 (0.007546) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.967258 (0.007939) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.962540 (0.008794) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.959175 (0.009461) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.949737 (0.011272) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.949737 (0.011172) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.940462 (0.011763) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.939149 (0.011991) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.914777 (0.013427) with: {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
0.916705 (0.013601) with: {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}


CLearly performance increase by more than 1% in cases of higher values of C

# 2. Applyng and tuning Ridge Classifier

In [36]:
from sklearn.linear_model import RidgeClassifier

# define models and parameters
model = RidgeClassifier()
alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# define grid search
grid = dict(alpha=alpha)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.944977 using {'alpha': 0.1}
0.944977 (0.007599) with: {'alpha': 0.1}
0.944977 (0.007599) with: {'alpha': 0.2}
0.944936 (0.007651) with: {'alpha': 0.3}
0.944936 (0.007651) with: {'alpha': 0.4}
0.944936 (0.007651) with: {'alpha': 0.5}
0.944936 (0.007651) with: {'alpha': 0.6}
0.944936 (0.007651) with: {'alpha': 0.7}
0.944936 (0.007651) with: {'alpha': 0.8}
0.944854 (0.007514) with: {'alpha': 0.9}
0.944854 (0.007514) with: {'alpha': 1.0}


Giving poor performance even less than simple Logistic regression

# 3. Applyng and tuning KNN Classifier

In [38]:
from sklearn.neighbors import KNeighborsClassifier

# define models and parameters
model = KNeighborsClassifier()
n_neighbors = range(1, 21, 2)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']

# define grid search
grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1.000000 using {'metric': 'manhattan', 'n_neighbors': 1, 'weights': 'uniform'}
0.999918 (0.000307) with: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}
0.999918 (0.000307) with: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'distance'}
0.999754 (0.000492) with: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}
0.999754 (0.000492) with: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
0.999139 (0.001277) with: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}
0.999467 (0.000989) with: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}
0.998195 (0.001790) with: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'uniform'}
0.999097 (0.001268) with: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'distance'}
0.998031 (0.001814) with: {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'uniform'}
0.998769 (0.001456) with: {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'distance'}
0.997005 (

Std deviation is 0 if we are using manhattan metric cases, lets see some more classifiers

# 4. Applyng and tuning Support Vector Classifier

In [24]:
from sklearn.svm import SVC

# define model and parameters
model = SVC()
kernel = ['poly', 'rbf', 'sigmoid']
C = [50, 10, 1.0, 0.1, 0.01]
gamma = ['scale']

# define grid search
grid = dict(kernel=kernel,C=C,gamma=gamma)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1.000000 using {'C': 50, 'gamma': 'scale', 'kernel': 'poly'}
1.000000 (0.000000) with: {'C': 50, 'gamma': 'scale', 'kernel': 'poly'}
1.000000 (0.000000) with: {'C': 50, 'gamma': 'scale', 'kernel': 'rbf'}
0.703552 (0.013984) with: {'C': 50, 'gamma': 'scale', 'kernel': 'sigmoid'}
1.000000 (0.000000) with: {'C': 10, 'gamma': 'scale', 'kernel': 'poly'}
1.000000 (0.000000) with: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
0.703388 (0.013959) with: {'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}
0.998400 (0.001562) with: {'C': 1.0, 'gamma': 'scale', 'kernel': 'poly'}
0.990645 (0.003506) with: {'C': 1.0, 'gamma': 'scale', 'kernel': 'rbf'}
0.582711 (0.110350) with: {'C': 1.0, 'gamma': 'scale', 'kernel': 'sigmoid'}
0.979566 (0.005287) with: {'C': 0.1, 'gamma': 'scale', 'kernel': 'poly'}
0.949450 (0.008016) with: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
0.522282 (0.015492) with: {'C': 0.1, 'gamma': 'scale', 'kernel': 'sigmoid'}
0.929960 (0.009142) with: {'C': 0.01, 'gamma': 'scale',

Performing very good in cases of higher values of C

# 5. Applyng and tuning Bagging Classifier

In [41]:
from sklearn.ensemble import BaggingClassifier

# define models and parameters
model = BaggingClassifier()
n_estimators = [10, 100, 1000]
# define grid search
grid = dict(n_estimators=n_estimators)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1.000000 using {'n_estimators': 10}
1.000000 (0.000000) with: {'n_estimators': 10}
1.000000 (0.000000) with: {'n_estimators': 100}
1.000000 (0.000000) with: {'n_estimators': 1000}


Giving best results in all the cases, also seems faster than others so Bagging Classifier is clearly the winner model

In [11]:
import pickle

oe = OrdinalEncoder()
X = df.drop(['class', 'veil-type'], axis=1)
y = df['class']
y = np.array(y).reshape(-1,1)

X = oe.fit_transform(X)

y = oe.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=12)

# define models and parameters
model = BaggingClassifier(n_estimators=10)
model.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BaggingClassifier()

In [46]:
yp = model.predict([[0, 2, 4, 0, 5, 0, 0, 0, 5, 0, 0, 2, 2, 5, 5, 1, 1, 4, 3, 4, 2]])
yp

array([0.])

In [47]:
yp[0]==0

True

In [10]:
# le = LabelEncoder()
# le.fit(processed_data[col])

# def data_preprocessing(data):
#     processed_data = data.drop('veil-type', axis=1)
#     for col in processed_data.columns:
#         processed_data[col] = le.transform(processed_data[col])

#     return processed_data


# def model_training(processed_df):
#     X = processed_df.drop('class', axis=1)
#     y = processed_df['class']

#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.33, random_state=102)

#     # define models and parameters
#     model = BaggingClassifier(n_estimators=10)
#     model.fit(X_train, y_train)

#     return model

In [8]:
specifications = {
'cap-shape': 'b',
'cap-surface':'s',
'cap-color':'w',
'bruises':'t',
'odor':'l',
'gill-attachment':'f',
'gill-spacing':'c',
'gill-size':'b',
'gill-color':'n',
'stalk-shape':'e',
'stalk-root':'c',
'stalk-surface-above-ring':'s',
'stalk-surface-below-ring':'s',
'stalk-color-above-ring':'w',
'stalk-color-below-ring':'w',
'veil-color':'w',
'ring-number':'o',
'ring-type':'p',
'spore-print-color':'n',
'population':'n',
'habitat':'m'}

In [49]:
sp = pd.DataFrame(specifications, index=[0])
sp

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,b,s,w,t,l,f,c,b,n,e,...,s,s,w,w,w,o,p,n,n,m


In [5]:
import pickle
oe1 = OrdinalEncoder()
X = df.drop(['class', 'veil-type'], axis=1)
X = oe1.fit_transform(X)
pickle.dump(oe1, open('oe1.pkl', 'wb'))

In [12]:
oe = pickle.load(open('oe1.pkl', 'rb'))
sample = pd.DataFrame(specifications, index=[0])   # or pd.DataFrame(specifications, index=[0])
# sample.drop('veil-type', axis=1, inplace=True)
processed_sample = oe1.transform(sample)
result = model.predict(processed_sample)

In [13]:
result

array([0.])